## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,42.91,70,0,20.71,clear sky
1,1,Dundee,GB,56.5000,-2.9667,52.59,82,0,3.44,clear sky
2,2,Igarka,RU,67.4667,86.5833,51.26,83,100,8.70,light rain
3,3,Tasiilaq,GL,65.6145,-37.6368,50.16,83,37,4.68,scattered clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,71.55,90,7,7.00,clear sky


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Atuona,PF,-9.8000,-139.0333,77.09,68,100,13.67,overcast clouds
9,9,Dabhol,IN,17.6000,73.1667,78.78,84,100,13.94,overcast clouds
14,14,Hilo,US,19.7297,-155.0900,82.13,73,20,8.05,few clouds
15,15,Vaini,TO,-21.2000,-175.2000,77.16,73,20,27.63,few clouds
17,17,Kavieng,PG,-2.5744,150.7967,83.14,69,31,2.19,light rain
18,18,Hami,CN,42.8000,93.4500,77.40,39,100,4.43,overcast clouds
25,25,Kapaa,US,22.0752,-159.3190,87.78,69,75,4.00,broken clouds
37,37,Kaohsiung,TW,22.6163,120.3133,78.75,97,90,15.01,thunderstorm with heavy rain
42,42,Sao Miguel Do Araguaia,BR,-13.2750,-50.1628,77.38,38,31,4.74,scattered clouds
43,43,Miyako,JP,39.6367,141.9525,79.65,82,86,6.44,overcast clouds


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                257
City                   257
Country                256
Lat                    257
Lng                    257
Max Temp               257
Humidity               257
Cloudiness             257
Wind Speed             257
Current Description    257
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                256
City                   256
Country                256
Lat                    256
Lng                    256
Max Temp               256
Humidity               256
Cloudiness             256
Wind Speed             256
Current Description    256
dtype: int64

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Atuona,PF,77.09,overcast clouds,-9.8000,-139.0333,
9,Dabhol,IN,78.78,overcast clouds,17.6000,73.1667,
14,Hilo,US,82.13,few clouds,19.7297,-155.0900,
15,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
17,Kavieng,PG,83.14,light rain,-2.5744,150.7967,
18,Hami,CN,77.40,overcast clouds,42.8000,93.4500,
25,Kapaa,US,87.78,broken clouds,22.0752,-159.3190,
37,Kaohsiung,TW,78.75,thunderstorm with heavy rain,22.6163,120.3133,
42,Sao Miguel Do Araguaia,BR,77.38,scattered clouds,-13.2750,-50.1628,
43,Miyako,JP,79.65,overcast clouds,39.6367,141.9525,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))